In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

ModuleNotFoundError: No module named 'numpy'

In [3]:
# Завантаження даних MNIST
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()

# Нормалізація даних
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Розмірність кодувального простору
encoding_dim = 32  # Задаємо розмірність кодованого представлення

# Вхідне зображення
input_img = Input(shape=(784,))

# "Закодована" представлення
encoded = Dense(encoding_dim, activation='relu')(input_img)

# Відновлення вихідного зображення
decoded = Dense(784, activation='sigmoid')(encoded)

# Модель автокодувальника
autoencoder = Model(input_img, decoded)

# Компіляція моделі
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Навчання автокодувальника
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Оцінка точності моделі на тестових даних
score = autoencoder.evaluate(x_test, x_test)
print('Test loss:', score)

# Застосування автокодувальника для зображень
decoded_imgs = autoencoder.predict(x_test)


11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/50
235/235 [==============================] - 4s 12ms/step - loss: 0.2762 - val_loss: 0.1894
Epoch 2/50
235/235 [==============================] - 2s 10ms/step - loss: 0.1705 - val_loss: 0.1533
Epoch 3/50
235/235 [==============================] - 2s 9ms/step - loss: 0.1440 - val_loss: 0.1329
Epoch 4/50
235/235 [==============================] - 2s 10ms/step - loss: 0.1277 - val_loss: 0.1204
Epoch 5/50
235/235 [==============================] - 2s 10ms/step - loss: 0.1178 - val_loss: 0.1126
Epoch 6/50
235/235 [==============================] - 2s 8ms/step - loss: 0.1112 - val_loss: 0.1073
Epoch 7/50
235/235 [==============================] - 2s 9ms/step - loss: 0.1065 - val_loss: 0.1031
Epoch 8/50
235/235 [==============================] - 3s 11ms/step - loss: 0.1029 - val_loss: 0.1000
Epoch 9/50
235/235 [==============================] - 3s 12ms/step - loss: 0.1001 - val_loss: 0.0978
Epoch 10/50
235/235 [========

In [ ]:
# Завантаження даних MNIST та попередня обробка
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Збудування згорткового автокодувальника
input_img = Input(shape=(28, 28, 1))

# Енкодер
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Декодер
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Створення моделі автокодувальника
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Навчання автокодувальника
history = autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

# Графік функції втрат
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Побудова результуючих зображень
decoded_imgs = autoencoder.predict(x_test)

# Візуалізація результатів
n = 10  # Кількість зображень, які ми хочемо показати
plt.figure(figsize=(20, 4))
for i in range(n):
    # Вихідне зображення
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Відновлене зображення
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()